# Import dependencies and libraries

In [ ]:
import os
import json
import pickle as pickle

import geopandas as gpd
import numpy as np

# for fetching osm data
import osmnx as ox

# for visualization
import matplotlib
import matplotlib.pyplot as plt

# trackintel -> the functions will be imported with full names
import trackintel as ti


In [ ]:
# options for better printing and visualizing

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

matplotlib.rcParams["figure.dpi"] = 300
matplotlib.rcParams["xtick.labelsize"] = 13
matplotlib.rcParams["ytick.labelsize"] = 13
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

%matplotlib inline

In [ ]:
# get the data dir from the config file -> default stored in Data/Geolife/Data folder

DATA_DIR = os.path.join("..", "paths.json")
with open(DATA_DIR) as json_file:
    CONFIG = json.load(json_file)

save_dir = os.path.join("..", CONFIG["data_dir"])

# Load Geolife data
Trackintel provides a function [read_geolife](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_geolife) to directly load [Geolife](https://www.microsoft.com/en-us/research/publication/geolife-gps-trajectory-dataset-user-guide/) dataset into the trackintel positionfixes. For the tutorial and interactive section, we provided you a sample dataset with 20 selected users from the geolife dataset. This can be directly loaded from the `save_dir`. 

In [ ]:

pfs, mode_labels = ti.io.read_geolife(save_dir, print_progress=True)

## Reading your own data
Trackintel provides an I/O module for accessing movement data and storing intermediate or final results in a file or database. Three methods for converting movement data with attached attribute information to Trackintel-compatible formats are provided:
1) reading from Pandas Dataframes and Geopandas Geodataframes (**recommended**)
2) reading and writing from CSV file formats
3) reading and storing from PostgreSQL databases with PostGIS extension. Check the [input/output](https://trackintel.readthedocs.io/en/latest/modules/io.html) module for more information. 

An important consideration before loading data is to find the appropriate movement level for your data. Depending on the semantics of the data, you should call different reading functions, e.g., [read_positionfixes_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_positionfixes_csv) or [read_staypoints_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_staypoints_csv). We provide a reference of the different data types in the figure below:

![Trackintel data support](prepare/trackintel_structure.png)

In [ ]:
# TODO: use ti.read_positionfixes_csv() or ti.read_staypoints_csv() for loading your data. Check the documentation for the required input parameters
# selected_pfs = ti.read_positionfixes_csv()

# Trajectory context integration with OSM data

Desired outcome with geolife sample data, where the x-axis is the location ids and the y-axis shows the frequency of observing a particular POI in its proximity:

![Context integration results](prepare/context.png)

## Generate staypoints and locations
Activity staypoints represent individual visits to places that are significant to the visitor. Trackintel models these significant places using the *location* class to enable the characterization of the place that is visited. While the information attached to staypoints is bound to the individual visit (e.g., the specific activity or the time of day), the semantics of locations are related to the place independent of the visit (e. g., land use or the opening hours of a shop). Therefore, we use location data to integrate with the POI context. This requires the functions [generate_staypoints](https://trackintel.readthedocs.io/en/latest/modules/preprocessing.html#trackintel.preprocessing.generate_staypoints) and [generate_locations](https://trackintel.readthedocs.io/en/latest/modules/preprocessing.html#trackintel.preprocessing.generate_locations).

In [ ]:
# TODO: generate staypoints with generate_staypoints()
pfs, sp = pfs.generate_staypoints(...)

# create activity flag based on duration
sp = sp.create_activity_flag(method="time_threshold", time_threshold=25)

# generate locations
## filter activity staypoints
sp = sp.loc[sp["is_activity"] == True]

# TODO: generate locations with generate_locations()
sp, locs = sp.generate_locations(...)

In [ ]:
# TODO: inspect locations with .head(), .columns
# locs.head()

## Extract POI from OSM and link with locations

We can make certain statements about the reason for a trip if we know what is in the area at the destination. For example, it makes sense that we go to a university district to study, to a residential area when we go home, or to an area with many restaurants and bars to meet friends.

### POI from OSM

Following this intuition, we extract POIs from OpenStreetMap and link them spatially with the locations. First we define a filter, defining that OSM objects with the attribute `amenity` or `shop` are extracted from the OSM data set. These tags are very widespread in the OSM data because they describe various objects in a relatively general way. You can find out more about the OSM tags at https://wiki.openstreetmap.org/wiki/DE:Key:amenity and https://taginfo.openstreetmap.org.

We use the library `osmnx` to download the newest osm data from the internet and extract certain subsets of the data as GeoDataFrame. `osmnx` is an open-source library that provides support for working with OpenStreetMap data. It enables straightforward fetching of OSM data and is still actively maintained by a group of GIS enthusiasts. Checkout the library [homepage](https://osmnx.readthedocs.io/en/stable/getting-started.html) and its [examples](https://github.com/gboeing/osmnx-examples/tree/main/notebooks).

Here we define the filter below for the POIs for you.

In [ ]:
custom_filter = {'amenity': True, 'shop': True}

# apply the filter
pois = ox.features_from_bbox(40.0920, 39.8594, 116.4410, 116.1152, custom_filter)

# we now want the 'node' types
pois = pois.iloc[pois.index.get_level_values('element_type') == "node"]

pois.shape

Use the function `explore`, which is provided by the `geopandas` library for `GeoDataFrames`.

In [ ]:
pois.explore()

We can now display this data on a map and inspect it again with `.head (...)`.

In [ ]:
# Gather info about POI type (combines the tag info from "amenity" and "shop").
pois["poi_type"] = pois["amenity"]
pois["poi_type"] = pois["poi_type"].fillna(pois["shop"])

# plot
ax = pois.plot(column='poi_type', markersize=3, figsize=(12,12), legend=True, legend_kwds=dict(loc='upper left', ncol=5, bbox_to_anchor=(1, 1)))
plt.show()

In [ ]:
# Inspect the DataFrame using .head(...), .columns, etc.
pois.columns

### Conversion to projected coordinate system, buffering and spatial join

In the next step, we want to change the coordinate system from WGS 1984 to EPSG:4479, as the latter allows us to quantify distances in meters. `geopandas` offers the function `.to_crs(...)` for this operation. Then create a buffer around the sample points with the function `.buffer(dist)` and use the `gpd.sjoin(...)` function to perform a spatial join between the buffered sample points and the POI. Look at the data with `.head(...)` and check if the results make sense.

In [ ]:
# Create buffer around locations. 
# TODO: define a buffer size, remember the unit is meters in projected coordinate systems 
epsilon = ...


# project the data into the coordinate system EPSG:4479.
locs_4479 = gpd.GeoDataFrame(locs.drop(columns="center"), crs="EPSG:4326", geometry="extent")
locs_4479 = locs_4479.to_crs('EPSG:4479')
pois_4479 = pois.to_crs('EPSG:4479')

locs_buffer = locs_4479.copy()

# Create a buffer around the locations.
locs_buffer['extent'] = locs_4479.buffer(epsilon)


# TODO: Join the two GeoDataFrames using geopanda's sjoin function (use how='left').
locs_pois = gpd.sjoin(...).reset_index()


In [ ]:
# examine locations
locs_pois.head(3)

We can now count how often a certain type of POI has occurred in the vicinity of a location and draw this in a figure. Attention: We only plot the first 20 locations for simplicity.

In [ ]:
poi_types_per_loc = locs_pois.groupby(['id', 'poi_type'])['id'].count().unstack('poi_type').head(20)
poi_types_per_loc.plot(kind='bar', stacked=True, figsize=(12, 8))
plt.legend(ncol=3, loc='center left', bbox_to_anchor=(1.04, 0.5))
plt.show()